In [1]:
# For text preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
# For topic modeling
from gensim import corpora
from gensim.models import LdaModel
import pandas as pd
# Download NLTK Resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Z4Teen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Z4Teen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Z4Teen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
df = pd.read_csv(r'C:\Users\Z4Teen\Desktop\Anaconda Jupiter\data\npr.csv', encoding='latin1')
documents = df['Article'].tolist()


C:\Users\Z4Teen\AppData\Local\Temp\ipykernel_28848\3373391506.py:1: DtypeWarning: Columns (139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'C:\Users\Z4Teen\Desktop\Anaconda Jupiter\data\npr.csv', encoding='latin1')


In [3]:
stop_words = set(stopwords.words('english')) # Create a set of English stopwords
lemmatizer = WordNetLemmatizer() # Initialize a WordNet lemmatizer
def preprocess_text(text):
    tokens = word_tokenize(text.lower()) # Tokenize the text into words and convert to lowercase
    tokens = [token for token in tokens if token.isalnum()] # Filter out non-alphanumeric tokens
    tokens = [token for token in tokens if token not in stop_words] # Remove stopwords from the tokens
    tokens = [lemmatizer.lemmatize(token) for token in tokens] # Lemmatize each token
    return tokens # Return the preprocessed tokens

preprocessed_documents = [preprocess_text(doc) for doc in documents] # Preprocess each document in the list
print(preprocessed_documents[0])

['washington', '2016', 'even', 'policy', 'bipartisan', 'politics', 'sense', 'year', 'show', 'little', 'sign', 'ending', 'president', 'obama', 'moved', 'sanction', 'russia', 'alleged', 'interference', 'election', 'concluded', 'republican', 'long', 'called', 'similar', 'severe', 'measure', 'could', 'scarcely', 'bring', 'approve', 'house', 'speaker', 'paul', 'ryan', 'called', 'obama', 'measure', 'also', 'prime', 'example', 'ineffective', 'foreign', 'policy', 'left', 'america', 'weaker', 'eye', 'gop', 'leader', 'sounded', 'much', 'theme', 'urging', 'president', 'obama', 'year', 'take', 'strong', 'action', 'deter', 'worldwide', 'aggression', 'including', 'operation', 'wrote', 'devin', 'nunes', 'chairman', 'house', 'intelligence', 'committee', 'week', 'left', 'office', 'president', 'suddenly', 'decided', 'stronger', 'measure', 'indeed', 'appearing', 'cnn', 'frequent', 'obama', 'critic', 'trent', 'frank', 'called', 'action', 'said', 'three', 'time', 'obama', 'found', 'meanwhile', 'fox', 'news

In [4]:
# Create a Gensim Dictionary object from the preprocessed documents
dictionary = corpora.Dictionary(preprocessed_documents)
# Filter out tokens that appear in less than 15 documents or more than 50% of the documents
dictionary.filter_extremes(no_below=15, no_above=0.5)
# Convert each preprocessed document into a bag-of-words representation using the dictionary
corpus = [dictionary.doc2bow(doc) for doc in preprocessed_documents]

In [ ]:
# Run LDA
lda_model = LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15) # Train an LDA model on the corpus with 2 topics using Gensim's LdaModel class

In [ ]:
# empty list to store dominant topic labels for each document
article_labels = []
# iterate over each processed document
for i, doc in enumerate(preprocessed_documents):
# for each document, convert to bag-of-words representation
bow = dictionary.doc2bow(doc)
# get list of topic probabilities
topics = lda_model.get_document_topics(bow)
# determine topic with highest probability
dominant_topic = max(topics, key=lambda x: x[1])[0]
# append to the list
article_labels.append(dominant_topic)
# Create DataFrame
df_result = pd.DataFrame({"Article": documents, "Topic": article_labels})
# Print the DataFrame
print("Table with Articles and Topic:")
print(df_result)
print()


In [ ]:
# Print top terms for each topic
for topic_id in range(lda_model.num_topics):
print(f"Top terms for Topic #{topic_id}:")
top_terms = lda_model.show_topic(topic_id, topn=10)
print([term[0] for term in top_terms])
print()


In [ ]:
# Print the top terms for each topic with weight
print("Top Terms for Each Topic:")
for idx, topic in lda_model.print_topics():
print(f"Topic {idx}:")
terms = [term.strip() for term in topic.split("+")]
for term in terms:
weight, word = term.split("*")
print(f"- {word.strip()} (weight: {weight.strip()})")
print()